In [1]:

from mdutils.mdutils import MdUtils
from mdutils import Html



def create_file(ds):
    
    data = get_datasource(URL,cook,ds['uuid'])
    
    naming = data['name'].replace('/','-')
    
    mdFile = MdUtils(file_name=naming)
    
    fields = []
    for any_schema in get_datasource(URL, cook, ds['uuid'])['schemas']:
        schemaID = any_schema['schemaId']
        structure = get_schema(URL,cook,schemaID)

        for field in structure['schema']:
            if field['columnName'] not in fields:
                fields.append(field['columnName'])
                

    schema = (' '.join(fields))
    title = data['name']
    organization = 'Lab'
    notes = 'Used in '+str(data['incomingLineages']+data['outgoingLineages'])+' lineage(s)'
    name = data['name']
    format = data['format']
    url = data['location']

    mdFile.write("---")
    mdFile.new_line("schema: chicago")
    mdFile.new_line("title: "+title)
    mdFile.new_line("organization: "+organization)
    mdFile.new_line("notes: "+notes)
    mdFile.new_line("resources:")
    mdFile.new_line('  - name: '+name+' \n    url: '+url+' \n    format : '+format)
    mdFile.new_line('schema_fields: '+schema)
    mdFile.new_line('category:'+'\n  - Loan Acceptance Product')
    mdFile.new_line('maintainer:'+' User')
    mdFile.new_line('maintainer_email:'+' UserMail')

    mdFile.new_line("---")
    mdFile.create_md_file()


In [2]:
def get_cookie(damurl,PAT):
    import requests
    session = requests.Session()
    cookieurl=damurl + '/api/auth/callback?client_name=ExternalAppTokenClient'
    header = {'X-External-App-Token':PAT}
    response = session.post(url=cookieurl,headers=header,verify=False)
    cookie = session.cookies
    return cookie

def get_datasources(url,cookie):
    import requests
    v = requests.get(url+"/api/services/v1/resources/datasources",cookies=cookie,verify=False)
    return v.json()

def get_datasource(url, cookie, dsId):
    import requests
    v = requests.get(url+"/api/services/v1/resources/datasource/%s" %dsId,cookies=cookie,verify=False)
    return v.json() 


def get_schema(url, cookie, schemaId):
    import requests
    v = requests.get(url+"/api/services/v1/resources/schema/%s" %schemaId,cookies=cookie,verify=False)
    return v.json() 



In [3]:
PAT = ""
URL = "  "

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
cook = get_cookie(URL,PAT)

In [5]:
dss = get_datasources(URL,cook)

In [6]:
project_lab = []
project_prod = []

for ds in dss:
    if 'LoanApproval' in ds['location'] and '2020' not in ds['location']:
        #print(ds['location'])
        project_lab.append(ds)
    elif 'LoanApproval' in ds['location'] and '2020' in ds['location']:
        #print(ds['location'])
        project_prod.append(ds)
    

In [7]:
prod = {}
for ds in project_prod:
    if ds['location'].split('/')[-1] not in prod.keys():
        prod[ds['location'].split('/')[-1]] = []
    prod[ds['location'].split('/')[-1]].append(ds)
  
        
        

In [8]:
for ds in project_lab:
    create_file(ds)

In [9]:
def create_file_multiple(ds):
    fields = []
    for el in ds:
    
        #data = get_datasource(URL,cook,ds['uuid'])

        naming = el

        mdFile = MdUtils(file_name=naming)

        for element in ds[el]:
        
            for any_schema in get_datasource(URL, cook, element['uuid'])['schemas']:
                schemaID = any_schema['schemaId']
                structure = get_schema(URL,cook,schemaID)

                for field in structure['schema']:
                    if field['columnName'] not in fields:
                        fields.append(field['columnName'])


        schema = (' '.join(fields))
        
        title = el
        
        organization = 'Production'
        
        income = 0
        outcome = 0
        
        for element in ds[el]:
            data = get_datasource(URL,cook,element['uuid'])
            income += data['incomingLineages']
            outcome+= data['outgoingLineages']
        notes = 'Used in '+str(income+outcome)+' lineage(s)'
        

        mdFile.write("---")
        mdFile.new_line("schema: chicago")
        mdFile.new_line("title: "+title)
        mdFile.new_line("organization: "+organization)
        mdFile.new_line("notes: "+notes)
        mdFile.new_line("resources:")
        
        
        
        for element in ds[el]:
            data = get_datasource(URL,cook,element['uuid'])
            name = data['name']
            format = data['format']
            url = data['location']
            
            mdFile.new_line('  - name: '+name+' \n    url: '+url+' \n    format : '+format)
        mdFile.new_line('schema_fields: '+schema)
        mdFile.new_line('category:'+'\n  - Loan Acceptance Product')
        mdFile.new_line('maintainer:'+' User')
        mdFile.new_line('maintainer_email:'+' UserMail')

        mdFile.new_line("---")
        mdFile.create_md_file()

In [10]:
create_file_multiple(prod)